In [1]:
library(ggplot2)
library(SingleR)
library(dplyr)
library(celldex)
library(RColorBrewer)

source("../tools/formating/formating.R")

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [ ]:
inupt_path <- "/ps/ai-ready/data/tung/tung.h5Seurat"

In [ ]:
srat <- load_seurat(inupt_path)
srat

In [ ]:
names(srat@assays)

In [ ]:
'RNA' %in% names(srat@assays)

In [ ]:
DefaultAssay(srat)<-'SCT'

In [ ]:
names(srat@assays)

In [ ]:
DefaultAssay(srat)

In [ ]:
get_assays_from_seurat <- function(seurat_object) {
    assays_names <- 'RNA'

    if (DefaultAssay(seurat_object) != 'RNA') {
        assays_names <- names(srat@assays)
    }
    assays_names
}

In [ ]:
assays_names <- get_assays_from_seurat(srat)
assays_names

In [ ]:
SaveH5Seurat(srat, filename = "/ps/ai-ready/data/tung/tung1.h5Seurat", overwrite = TRUE, verbose = FALSE)

In [ ]:
seu = DietSeurat(
  srat,
  counts = TRUE, # so, raw counts save to adata.layers['counts']
  data = TRUE, # so, log1p counts save to adata.X when scale.data = False, else adata.layers['data']
  scale.data = FALSE, # if only scaled highly variable gene, the export to h5ad would fail. set to false
  features = rownames(srat), # export all genes, not just top highly variable genes
  assays = "RNA",
  dimreducs = c("pca","umap"),
  graphs = c("RNA_nn", "RNA_snn"), # to RNA_nn -> distances, RNA_snn -> connectivities
  misc = TRUE
)

In [ ]:
seu[['RNA']]@counts

In [ ]:
seu[['RNA']]

In [ ]:
path <- MuDataSeurat::WriteH5AD(seu, "/ps/ai-ready/data/tung/seu.h5ad", assay="RNA")
path

In [ ]:
DefaultAssay(srat)<-'SCT'

seu_sct = DietSeurat(
  srat,
  counts = TRUE, # so, raw counts save to adata.layers['counts']
  data = TRUE, # so, log1p counts save to adata.X when scale.data = False, else adata.layers['data']
  scale.data = FALSE, # if only scaled highly variable gene, the export to h5ad would fail. set to false
  features = rownames(srat), # export all genes, not just top highly variable genes
  assays = "SCT",
  dimreducs = c("pca","umap"),
  graphs = c("RNA_nn", "RNA_snn"), # to RNA_nn -> distances, RNA_snn -> connectivities
  misc = TRUE
)
seu_sct

In [ ]:
seu_sct[['SCT']]@counts

In [ ]:
DefaultAssay(seu_sct)<-'RNA'

In [ ]:
path <- "/ps/ai-ready/data/tung/umi.h5Seurat"
seurat_object <- LoadH5Seurat(path)
seurat_object

In [ ]:
DefaultAssay(seurat_object)

In [4]:
path <- "/ps/ai-ready/data/tung/umi.h5Seurat"
results <- convert_seurat_to_anndata(path, assay = 'RNA')

Validating h5Seurat file

Initializing ERCC with data

Adding counts for ERCC

Adding miscellaneous information for ERCC

Initializing RNA with data

Adding counts for RNA

Adding feature-level metadata for RNA

Adding miscellaneous information for RNA

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results

Validating h5Seurat file



In [ ]:
# inupt_path <- "/ps/ai-ready/data/filtered_gene_bc_matrices/hg19/"
inupt_path <- "/ps/ai-ready/data/kbcfh/Anndata/hca_heart_neuronal_raw.h5ad"
path_of_scrublet_calls<-"/ps/ai-ready/data/filtered_gene_bc_matrices/hg19/scrublet_calls.tsv"

In [ ]:
srat <- load_seurat(inupt_path)
srat

In [ ]:
default_assay <- "RNA"

In [ ]:
if(!default_assay %in% names(x = srat)) stop(paste(default_assay, "does not exist."))
DefaultAssay(object = srat) <-default_assay

In [ ]:
head(srat[[]])

In [ ]:
srat[[default_assay]]@counts

In [ ]:
if(!paste0("nCount_", default_assay) %in% names(x = srat[[]])) srat[[paste0("nCount_", default_assay)]] <- colSums(x = srat[[default_assay]], slot = "counts")  # nCount of the default assay
if(!paste0("nFeature_", default_assay) %in% names(x = srat[[]])) srat[[paste0("nFeature_", default_assay)]] <- colSums(x = GetAssayData(object = srat[[default_assay]], slot = "counts") > 0)  # nFeature of the default assay

In [ ]:
meta <- srat@meta.data
dim(meta)
head(meta)

summary(meta[paste0("nCount_", default_assay)])
summary(meta[paste0("nFeature_", default_assay)])

In [ ]:
srat[["percent.mt"]] <- PercentageFeatureSet(srat, pattern = "^MT-") # Michochondrial genes
srat[["percent.rb"]] <- PercentageFeatureSet(srat, pattern = "^RP[SL]") # Ribosomal proteins (their names begin with RPS or RPL

In [ ]:
# add the doublet annotation
doublets <- read.table(path_of_scrublet_calls, header = F, row.names = 1)
colnames(doublets) <- c("Doublet_score", "Is_doublet")
srat <- AddMetaData(srat, doublets)
head(srat[[]])

In [ ]:
names(srat[[]])

In [ ]:
srat[[]]["scrublet_score"]

In [ ]:
doublets

In [ ]:
VlnPlot(srat, features = c(paste0("nFeature_", default_assay), paste0("nCount_", default_assay),"percent.mt","percent.rb"),ncol = 4,pt.size = 0.1) & 
  theme(plot.title = element_text(size=10))

In [ ]:
FeatureScatter(srat, feature1 = paste0("nCount_", default_assay), feature2 = "percent.mt")
FeatureScatter(srat, feature1 = paste0("nCount_", default_assay), feature2 = paste0("nFeature_", default_assay))
FeatureScatter(srat, feature1 = paste0("nCount_", default_assay), feature2 = "percent.rb")
FeatureScatter(srat, feature1 = "percent.rb", feature2 = "percent.mt")
# FeatureScatter(srat, feature1 = paste0("nFeature_", default_assay), feature2 = "Doublet_score")
FeatureScatter(srat, feature1 = paste0("nFeature_", default_assay), feature2 = "scrublet_score")

In [ ]:
srat[['QC']] <- ifelse(srat@meta.data$Is_doublet == 'True','Doublet','Pass')
srat[['QC']] <- ifelse(srat@meta.data[paste0("nFeature_", default_assay)] < 500 & srat@meta.data$QC == 'Pass','Low_nFeature',srat@meta.data$QC)
srat[['QC']] <- ifelse(srat@meta.data[paste0("nFeature_", default_assay)] < 500 & srat@meta.data$QC != 'Pass' & srat@meta.data$QC != 'Low_nFeature',paste('Low_nFeature',srat@meta.data$QC,sep = ','),srat@meta.data$QC)
srat[['QC']] <- ifelse(srat@meta.data$percent.mt > 15 & srat@meta.data$QC == 'Pass','High_MT',srat@meta.data$QC)
srat[['QC']] <- ifelse(srat@meta.data[paste0("nFeature_", default_assay)] < 500 & srat@meta.data$QC != 'Pass' & srat@meta.data$QC != 'High_MT',paste('High_MT',srat@meta.data$QC,sep = ','),srat@meta.data$QC)
table(srat[['QC']])

In [ ]:
srat[[]]

In [ ]:
VlnPlot(srat, features = c("nFeature_RNA","nCount_RNA","percent.mt","percent.rb"),ncol = 4,pt.size = 0.1) & 
  theme(plot.title = element_text(size=10))

In [ ]:
FeatureScatter(srat, feature1 = "nCount_RNA", feature2 = "percent.mt")
FeatureScatter(srat, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
FeatureScatter(srat, feature1 = "nCount_RNA", feature2 = "percent.rb")
FeatureScatter(srat, feature1 = "percent.rb", feature2 = "percent.mt")
FeatureScatter(srat, feature1 = "nFeature_RNA", feature2 = "Doublet_score")

In [ ]:
srat[['QC']] <- ifelse(srat@meta.data$Is_doublet == 'True','Doublet','Pass')
srat[['QC']] <- ifelse(srat@meta.data$nFeature_RNA < 500 & srat@meta.data$QC == 'Pass','Low_nFeature',srat@meta.data$QC)
srat[['QC']] <- ifelse(srat@meta.data$nFeature_RNA < 500 & srat@meta.data$QC != 'Pass' & srat@meta.data$QC != 'Low_nFeature',paste('Low_nFeature',srat@meta.data$QC,sep = ','),srat@meta.data$QC)
srat[['QC']] <- ifelse(srat@meta.data$percent.mt > 15 & srat@meta.data$QC == 'Pass','High_MT',srat@meta.data$QC)
srat[['QC']] <- ifelse(srat@meta.data$nFeature_RNA < 500 & srat@meta.data$QC != 'Pass' & srat@meta.data$QC != 'High_MT',paste('High_MT',srat@meta.data$QC,sep = ','),srat@meta.data$QC)
table(srat[['QC']])

In [ ]:
srat[['QC']]

In [ ]:
VlnPlot(subset(srat, subset = QC == 'Pass'), 
        features = c("nFeature_RNA", "nCount_RNA", "percent.mt","percent.rb"), ncol = 4, pt.size = 0.1) & 
  theme(plot.title = element_text(size=10))